In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

#categorizing the input and output data
bat_data = pd.read_csv('TrainingData.csv')
train_bat = bat_data

#Find a better way to categorize working ion
#not just a blank random scale
train_bat = train_bat.replace('Li', 1)
train_bat = train_bat.replace('Ca', 2)
train_bat = train_bat.replace('Cs', 3)
train_bat = train_bat.replace('Rb', 4)
train_bat = train_bat.replace('K', 5)
train_bat = train_bat.replace('Y', 6)
train_bat = train_bat.replace('Na', 7)
train_bat = train_bat.replace('Al', 8)
train_bat = train_bat.replace('Zn', 9)
train_bat = train_bat.replace('Mg', 0)

train_bat = train_bat.replace('Orthorombic', 1)
train_bat = train_bat.replace('Monoclinic', 2)
train_bat = train_bat.replace('Trigonal', 3)
train_bat = train_bat.replace('Triclinic', 4)
train_bat = train_bat.replace('Tetragonal', 5)
train_bat = train_bat.replace('Hexagonal', 6)
train_bat = train_bat.replace('Cubic', 7)

D_in, H, H2, D_out, N = 91, 100, 75, 2, 4000
dtype = torch.float
device = torch.device('cpu')


#define testing sets
x_train = train_bat
x_train = x_train.drop(columns=['Battery ID', 'Gravimetric Capacity (units)', 'Volumetric Capacity', 'Max Delta Volume'])

y_train = train_bat[['Gravimetric Capacity (units)', 'Volumetric Capacity']]

#test-train split
#optimize for user data
x_test = x_train[4000:]
y_test = y_train[4000:]

x_train = x_train[:4000]
y_train = y_train[:4000]

#Defining training and testing data
x_train_np = np.array(x_train)
x_train_torch = torch.tensor(x_train_np, device = device, dtype = dtype)

y_train_np = np.array(y_train)
y_train_torch = torch.tensor(y_train_np, device = device, dtype = dtype)

x_test_np = np.array(x_test)
x_test_torch = torch.tensor(x_test_np, device = device, dtype = dtype)

y_test_np = np.array(y_test)
y_test_torch = torch.tensor(y_test_np, device = device, dtype = dtype)

#Defining weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)
#w3 = torch.randn(H2, D_out, device=device, dtype=dtype)

#define model
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H, H2),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H2, D_out),
    #nn.Softmax(dim=1) #normalizing the data,
)

optimizer = optim.SGD(model.parameters(), lr=1e-10)
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-10
for t in range(20000):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x_train_torch)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y_train_torch)
    if t % 1000 == 999:
        print(t, loss.item())

    # Zero the gradients before running the backward/training pass.
    optimizer.zero_grad()
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()
    optimizer.step()
    
    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad


999 679410048.0
1999 402594400.0
2999 329905312.0
3999 296787072.0
4999 234559776.0
5999 240641280.0
6999 225287488.0
7999 206256992.0
8999 199251984.0
9999 208449312.0
10999 189687568.0
11999 155069488.0
12999 174656640.0
13999 170824320.0
14999 158934256.0
15999 131600992.0
16999 132209696.0
17999 134869280.0
18999 131025392.0
19999 131932648.0


In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

#categorizing the input and output data
bat_data = pd.read_csv('NEWTrainingData_StandardScaler.csv')
train_bat = bat_data

D_in, H, H2, D_out, N = 165, 100, 75, 2, 4000
dtype = torch.float
device = torch.device('cpu')


#define testing sets
x_train = train_bat.drop(columns=['Unnamed: 0', 'Gravimetric Capacity (units)', 'Volumetric Capacity', 'Max Delta Volume'])
y_train = train_bat[['Gravimetric Capacity (units)', 'Volumetric Capacity']]

#test-train split
#optimize for user data
x_test = x_train[4000:]
y_test = y_train[4000:]

x_train = x_train[:4000]
y_train = y_train[:4000]

#Defining training and testing data
x_train_np = np.array(x_train)
x_train_torch = torch.tensor(x_train_np, device = device, dtype = dtype)

y_train_np = np.array(y_train)
y_train_torch = torch.tensor(y_train_np, device = device, dtype = dtype)

x_test_np = np.array(x_test)
x_test_torch = torch.tensor(x_test_np, device = device, dtype = dtype)

y_test_np = np.array(y_test)
y_test_torch = torch.tensor(y_test_np, device = device, dtype = dtype)

#Defining weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)
#w3 = torch.randn(H2, D_out, device=device, dtype=dtype)

#define model
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H, H2),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H2, D_out),
    #nn.Softmax(dim=1) #normalizing the data,
)

optimizer = optim.SGD(model.parameters(), lr=1e-5)
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-5
for t in range(10000):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x_train_torch)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y_train_torch)
    if t % 1000 == 999:
        print(t, loss.item())

    # Zero the gradients before running the backward/training pass.
    optimizer.zero_grad()
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()
    optimizer.step()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad


999 315.8780822753906
1999 247.41452026367188
2999 290.8736572265625
3999 182.211669921875
4999 190.3646697998047
5999 177.3671112060547
6999 155.70242309570312
7999 140.84320068359375
8999 127.96055603027344
9999 127.28811645507812


Now for our testing error:

In [47]:
y_pred_test = model(x_test_torch)
loss_fn(y_pred_test, y_test_torch)

tensor(1059.3948, grad_fn=<MseLossBackward>)

In [2]:
import pandas as pd

In [6]:
bat_data = pd.read_csv('NEWTrainingData_MinMaxScaler.csv')
bat_data.drop(columns='Unnamed: 0')

bat_data.sample(frac=1)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,108,109,110,111,112,113,114,Gravimetric Capacity (units),Volumetric Capacity,Max Delta Volume
3170,3170,-0.021439,0.434442,0.232006,-0.313235,0.662584,-0.383811,-0.321362,-0.410063,0.793190,...,0.011836,-0.064973,-0.026135,-0.036112,-0.103705,-0.089368,-0.051041,0.065541,0.064827,0.000090
1261,1261,0.528843,-0.939930,-0.472162,0.642015,0.790263,-0.257633,0.444864,0.115337,-0.504389,...,0.009291,0.005186,-0.003646,0.005476,0.009730,0.004836,0.008583,0.032678,0.033585,0.000014
1858,1858,0.048449,-0.761757,-0.315581,0.445586,0.687430,0.000507,0.288167,-0.386637,0.908315,...,0.010372,-0.006741,-0.018359,0.007269,0.002286,0.002886,-0.002976,0.098209,0.088453,0.000175
159,159,-0.274586,-0.720130,0.166500,-0.727113,0.176757,0.215055,-0.106825,-0.474195,-0.021725,...,-0.010487,-0.003466,-0.000494,-0.003743,0.002878,0.002816,-0.001719,0.045741,0.052531,0.000009
2466,2466,1.759942,-0.051818,-0.351541,0.430430,0.118720,-0.516305,0.966642,0.055359,-0.553396,...,-0.005964,0.002237,-0.000175,0.015527,0.015417,-0.002032,0.005547,0.028856,0.050987,0.000213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3285,3285,0.339362,0.797677,0.123802,-0.664653,0.432301,-0.002011,-0.266529,-0.516055,0.076350,...,-0.012423,-0.002557,-0.012378,-0.029732,0.035415,0.006042,0.002168,0.052925,0.094199,0.000118
3550,3550,1.640616,1.534770,0.550321,0.802014,-0.393971,1.300032,0.498085,-1.003063,-0.271052,...,0.026294,0.182483,-0.017186,-0.084999,-0.075288,-0.038355,-0.025104,0.193584,0.436468,0.009519
1727,1727,-0.170767,-0.275342,-1.177806,0.636252,-0.607178,0.005092,0.564509,-0.246154,-0.176839,...,-0.006973,0.009637,0.013091,-0.012385,0.012166,-0.002393,0.011525,0.048004,0.069435,0.000102
2659,2659,1.375867,0.439795,0.339684,1.589268,0.095903,0.592655,-0.330470,-0.232737,0.595775,...,-0.040433,-0.051931,-0.003206,-0.000549,-0.014434,0.009896,0.005222,0.171171,0.178901,0.009579


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

#categorizing the input and output data
bat_data = pd.read_csv('NEWTrainingData_MinMaxScaler.csv')
train_bat = bat_data

D_in, H, H2, D_out, N = 115, 100, 75, 2, 4000
dtype = torch.float
device = torch.device('cpu')


#define testing sets
x_train = train_bat.drop(columns=['Unnamed: 0', 'Gravimetric Capacity (units)', 'Volumetric Capacity', 'Max Delta Volume'])
y_train = train_bat[['Gravimetric Capacity (units)', 'Volumetric Capacity']]

#shuffle data
x_train = x_train.sample(frac=1)
y_train = y_train.sample(frac=1)

#test-train split
#optimize for user data
x_test = x_train[4000:]
y_test = y_train[4000:]

x_train = x_train[:4000]
y_train = y_train[:4000]

#Defining training and testing data
x_train_np = np.array(x_train)
x_train_torch = torch.tensor(x_train_np, device = device, dtype = dtype)

y_train_np = np.array(y_train)
y_train_torch = torch.tensor(y_train_np, device = device, dtype = dtype)

x_test_np = np.array(x_test)
x_test_torch = torch.tensor(x_test_np, device = device, dtype = dtype)

y_test_np = np.array(y_test)
y_test_torch = torch.tensor(y_test_np, device = device, dtype = dtype)

#Defining weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)
#w3 = torch.randn(H2, D_out, device=device, dtype=dtype)

#define model
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H, H2),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H2, D_out),
    #nn.Softmax(dim=1) #normalizing the data,
)

optimizer = optim.SGD(model.parameters(), lr=1e-4)
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(1000):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x_train_torch)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y_train_torch)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward/training pass.
    optimizer.zero_grad()
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()
    optimizer.step()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad


99 39.268272399902344
199 39.03468322753906
299 38.86058044433594
399 38.703704833984375
499 38.548423767089844
599 38.38684844970703
699 38.21562576293945
799 38.02910232543945
899 37.825374603271484
999 37.60260772705078


In [32]:
mean=model(x_test_torch).mean()

In [26]:
y_pred_test = model(x_test_torch)
loss_fn(y_test_torch, y_pred_test)

tensor(4.9638, grad_fn=<SumBackward0>)

Look into removing volume change as an output
change number of inputs
Optimizers?
- Change atomic number of the working ion
- Electronegativity
- Ion charge
Use the standard scaler to optimize information